# IMPORT

In [57]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.oauth2 as oauth2
import json
import pandas as pd
import numpy as np
import requests
import re
import os
import datetime
from datetime import date, timedelta
from datetime import datetime
import glob
import time 

In [5]:
pd.set_option('display.max_columns', None)

# AUTHENTIFICATION
* guide for authentification with Spotify API: https://developer.spotify.com/documentation/general/guides/authorization-guide/
* calling personal credentials from environment https://able.bio/rhett/how-to-set-and-get-environment-variables-in-python--274rgt5
* open in terminal: touch ~/.bash_profile; open ~/.bash_profile
* implement: source ~/.bash_profile
* get new token https://developer.spotify.com/console/put-playlist/


#### Create and refresh token
run this to  refresh the token. Token runs out every 3600 seconds.

In [ ]:
my_client_id=os.getenv('spoti_CLIENT_ID')
my_client_secret=os.getenv('spoti_CLIENT_SECRET')

token = SpotifyClientCredentials(client_id=my_client_id, client_secret=my_client_secret)

#token to pull publicly available data like music analysis data
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

#token to pull songs of an individual usersp_pers=spotipy.Spotify(cache_token_personal)
cache_token_personal= '' 
sp1=spotipy.Spotify(cache_token_personal)
print(cache_token_personal)
print(cache_token)

# CRAWLING FOR CHARTS
HERE WE DOWNLOAD CSVs FORM www.spotifycharts.com

### Function
downloads CSVs from spotifycharts.com given the countries of choice and the time interval set below

* Note: this depends on the availability of the CSVs on spoitifycharts.com . Error may occure if we are trying to pull data for a day spotify does not provide charts data for. Slightly adapting frequency or start and end date might shift you around thos "data holes"

In [154]:
#DOWNLOAD LOOP FOR COUNTRY AND FOR TIME PERIOD
# prepare a folder named 'data' with two subfolders named 'viral' and 'regional'
def bulk_charts_download(country_charts, date_list):
    for country in country_charts:
        for date in date_list:
            url= f'https://spotifycharts.com/{what_charts}/{country}/daily/{date}/download'
            r = requests.get(url, allow_redirects=True)
            open(f'data/{what_charts}/{date}-{country}-{what_charts}.csv', 'wb').write(r.content)

### SET PARAMETERS FOR CRAWLING

#### Set Start and End Date

In [8]:
startdate = date(2018, 1, 1)   # start date
enddate = date(2020,12,30)   # end date
delta = enddate - startdate       # as timedelta

date_list=[]

for i in range(delta.days+1):
    day = startdate + timedelta(days=i)
    date_list.append(day.strftime("%Y-%m-%d"))

#### Chose frequency
between start date and end date how often do we pull charts data

In [9]:
date_list=date_list[0::30] #every 30 days

#### Country Codes

In [10]:
#enter list of country codes
country_charts=['us','br','jp','de','ca','gb','tr','is','mx','za','au','ar','eg','es','it','ma','nz','id','ru','ua','in','']
#HELP: LIST OF COUNTRY CHARTS
print(sp.country_codes)

#### Which charts are we crawling
Options: viral charts or regional top200

In [56]:
# VIRAL CHART sample URL https://spotifycharts.com/viral/us/daily/2020-03-13/download
# thats how i want filename to be safed regional-us-weekly-2020-03-13--2020-03-20.csv
# enter as string 'viral' for viral top 50 or 'regional' for local top 200
what_charts='viral'

#### Trigger download CSVs to folder: Calls function with parameters set above

In [360]:
bulk_charts_download(country_charts, date_list)

# CREATE DATAFRAMES

#### create dataframe from csvs
* intermediate step before adding music features

#### FUNCTION CSVs to Data Frames
* additionally extracts data on country, date and song IDs form the original URLs


In [16]:
def csv_to_dataframe(what_charts):
    path = f'data/{what_charts}/'                     # use your path
    all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

    df = [] # pd.concat takes a list of dataframes as an agrument
    for csv in all_files:
        
        # if condition since csv of top200 look differently than csvs from viral charts.
        # top 200 sharts do have a disclaimer not in the first line that messes with our columns upon import
        # we need to igonre it
        if what_charts=='regional': 
            frame = pd.read_csv(csv,skiprows=[0],error_bad_lines=False, warn_bad_lines=True)
            frame=frame.drop('Streams',axis=1)
        #this is the read in for viral charts csvs
        else:
            frame = pd.read_csv(csv,skiprows=[],error_bad_lines=False, warn_bad_lines=True)
        
        frame['filename'] = os.path.basename(csv)
        df.append(frame)
        
    #concat list of dataframes into 1
    df=pd.concat(df, ignore_index=True) 
    
    #finetuning columns with regex
    df['ID']= re.findall(r'\d\w+',str(list(df['URL']))) #extract ID from URL
    df['date']=re.findall(r'\d\d\d\d-\d\d-\d\d', str(list(df['filename'])))
    df['date']=pd.to_datetime(df.date, format="%Y-%m-%d", yearfirst=True)
    df['country']=re.findall(r'-[a-z][a-z]-', str(list(df['filename'])))
    df['country']=re.findall(r'[a-z][a-z]',str(list(df['country'])))
    
    # indicates if song is from viral charts or top 200 charts/ 'regional' indicates its part of  regional top200 charts hence we label it 'top200'
    df['group']=np.where(what_charts=='regional','top200',what_charts) 
    return df

In [17]:
viral_df_basic=csv_to_dataframe('viral')#pass in bracket if 'viral' or 'regional'
len(viral_df_basic)

33200

# SUBSETTING VIRAL CHARTS
* to make the project feasible for my processing means I need to subset
* my machine is able to make approx 6300 calls to the api per hour
* subsetting to the viral top 20 of 22 countries between 01/01/2017 and 31/12/2020 at 12 sample dates
* ending up with almost 5872 ids to represent 13827 entries

In [19]:
# unique song id: plenty of songs are in teh charts multiple time for different countries and at different dates
# lets see how many IDs are unique since we dont want to pull features for the same song multiple times
unique_ids=list(viral_df_basic.ID.unique())
len(unique_ids)

13827

In [21]:
# lets return all unique IDs of songs that are at one point place in the top20 viral charts in any country at any time

unique_ids_of_top20viral=set(viral_df_basic['ID'][viral_df_basic['ID'].isin (unique_ids) & viral_df_basic['Position'].isin(list(range(1,21)))])
len(unique_ids_of_top20viral)

5872

In [22]:
# using the top 20 of each countries viral charts at all sample points results in 5872 unique IDs
# thats below my 6300 ongs threashhold

In [23]:
#
viral_df_basic_top20=viral_df_basic[viral_df_basic['Position'].isin(list(range(1,21)))]
viral_df_basic_top20.shape 

(13280, 9)

In [24]:
len(viral_df_basic_top20.ID.unique())

5872

# INSPECT DATAFRAME

In [25]:
viral_df_basic_top20.head()

,Position,Track Name,Artist,URL,filename,ID,date,country,group
0,1,Sucker,Jonas Brothers,https://open.spotify.com/track/4y3OI86AEP6PQoD...,2019-03-27-in-viral.csv,4y3OI86AEP6PQoDE6olYhO,2019-03-27,in,viral
1,2,Undrunk,FLETCHER,https://open.spotify.com/track/5SHhPFh68OhUmuR...,2019-03-27-in-viral.csv,5SHhPFh68OhUmuRPymKX9d,2019-03-27,in,viral
2,3,Don't Call Me Up,Mabel,https://open.spotify.com/track/5WHTFyqSii0lmT9...,2019-03-27-in-viral.csv,5WHTFyqSii0lmT9R21abT8,2019-03-27,in,viral
3,4,Moonlight,Gaullin,https://open.spotify.com/track/0A5gdlrpAuQqZ2i...,2019-03-27-in-viral.csv,0A5gdlrpAuQqZ2iFgnqBFW,2019-03-27,in,viral
4,5,i'm so tired...,Lauv,https://open.spotify.com/track/7COXchtUOMd6uIT...,2019-03-27-in-viral.csv,7COXchtUOMd6uIT6HvmRaI,2019-03-27,in,viral


In [26]:
#are their songs that are in the charts of muliple countries ?
pd.pivot_table(viral_df_basic_top20,index=["Track Name",'Artist'], values=['country','ID'], aggfunc={'country':np.sum,'ID':'count'}).sort_values('ID',ascending=False).head(5)

,,ID,country
Track Name,Artist,,
Dance Monkey,Tones And I,89,deesdemxnzmxbresidnzegmamaesegbrbrmxidtraregnz...
death bed (coffee for your head),Powfu,61,jpitzazaitarisusaugbcausgbruaritjpzacaauusisgb...
Roses - Imanbek Remix,SAINt JHN,50,itzacaitarisusaugbcaisaucausgbruitgbcaaunzisgb...
Sweet but Psycho,Ava Max,50,zaarititaritzaauususgbaugbzadejpjpgbnzauismaca...
ily (i love you baby) (feat. Emilee),Surf Mesa,50,itzacaauitisarusgbisusgbgbusjpzacaauauuaitcais...


In [27]:
#anything empty?
viral_df_basic_top20.isnull().sum()

Position       0
Track Name    11
Artist        12
URL            0
filename       0
ID             0
date           0
country        0
group          0
dtype: int64

In [28]:
#NAN values
viral_df_basic_top20[viral_df_basic_top20['Artist'].isnull()]

,Position,Track Name,Artist,URL,filename,ID,date,country,group
2650,1,NaN,NaN,https://open.spotify.com/track/47rbjDud83d3aqv...,2019-04-26-in-viral.csv,47rbjDud83d3aqvbcTssei,2019-04-26,in,viral
6963,14,NaN,NaN,https://open.spotify.com/track/4LAgGxPsaI4HeOJ...,2019-11-22-mx-viral.csv,4LAgGxPsaI4HeOJStF8tgV,2019-11-22,mx,viral
8308,9,NaN,NaN,https://open.spotify.com/track/2IHTNejiUvoSTsF...,2020-10-17-mx-viral.csv,2IHTNejiUvoSTsFpovaESS,2020-10-17,mx,viral
8314,15,NaN,NaN,https://open.spotify.com/track/4xioHVjM8WJmLEV...,2020-10-17-mx-viral.csv,4xioHVjM8WJmLEVd5DD8BQ,2020-10-17,mx,viral
8316,17,NaN,NaN,https://open.spotify.com/track/6eiVcW005z9GU9D...,2020-10-17-mx-viral.csv,6eiVcW005z9GU9DpoEdlqh,2020-10-17,mx,viral
12211,12,NaN,NaN,https://open.spotify.com/track/2AgPxySIeAnXDHZ...,2019-06-25-es-viral.csv,2AgPxySIeAnXDHZvAQHg0R,2019-06-25,es,viral
14816,17,NaN,NaN,https://open.spotify.com/track/47rbjDud83d3aqv...,2019-05-26-in-viral.csv,47rbjDud83d3aqvbcTssei,2019-05-26,in,viral
15009,10,NO GOOD,NaN,https://open.spotify.com/track/4Qnz8tARYhUtDNe...,2020-07-19-jp-viral.csv,4Qnz8tARYhUtDNePNsLx2C,2020-07-19,jp,viral
18510,11,NaN,NaN,https://open.spotify.com/track/461JKAn7H6Sbx0q...,2020-12-16-in-viral.csv,461JKAn7H6Sbx0ql9IvRUG,2020-12-16,in,viral
19908,9,NaN,NaN,https://open.spotify.com/track/60pysSgEslc7i5b...,2020-05-20-br-viral.csv,60pysSgEslc7i5blU5zZbS,2020-05-20,br,viral


In [29]:
# how many songs per country 
viral_df_basic_top20['country'].value_counts()


it    740
nz    740
us    740
au    740
tr    740
es    740
id    740
ca    740
gb    740
jp    740
mx    740
br    720
ar    700
za    680
de    680
is    660
eg    520
ma    520
in    420
ua    120
ru    120
Name: country, dtype: int64

# CALL API FOR MUSIC FEATURES

### FUNCTION: GET MUSIC FEATURES FROM API

In [30]:
def getTrackFeatures(df):
    feature_names=['ID','duration_ms', 'popularity', 'acousticness','danceability', 'energy', 'key','instrumentalness', 'liveness', 'loudness', 'mode','speechiness', 'valence', 'tempo','loud_start']
    features_df=pd.DataFrame(columns=feature_names)
    
    causing_error=[]
    
    for id in set(df['ID']):#only loop through the unique IDs to enhance performance
        try:
            print(id)
            meta = sp.track(id)
            features = sp.audio_features(id)
            analysis= sp.audio_analysis(id) #does not seem to be available at all songs

            #collect in dictionary
            features_dict = {}   

        
            # meta
            features_dict['ID']=meta['id']
            features_dict['duration_ms'] = meta['duration_ms']
            features_dict['popularity'] = meta['popularity']
            # features
            features_dict['acousticness'] = features[0]['acousticness']
            features_dict['danceability'] = features[0]['danceability']
            features_dict['energy'] = features[0]['energy']
            features_dict['key'] = features[0]['key']
            features_dict['instrumentalness'] = features[0]['instrumentalness']
            features_dict['liveness'] = features[0]['liveness']
            features_dict['loudness'] = features[0]['loudness']
            features_dict['mode'] = features[0]['mode']
            features_dict['speechiness'] = features[0]['speechiness']
            features_dict['valence']=features[0]['valence']
            features_dict['tempo'] = features[0]['tempo']
            #analysis
            features_dict['loud_start']=analysis['sections'][0]['loudness']#loundness at the start
        except:
            causing_error.append(id)
            print('causing error: ',causing_error)
            
        #wrap up of all gathered features
        track_df = pd.DataFrame(features_dict,index=[0],columns=features_dict.keys())
        features_df = pd.concat([features_df, track_df], axis=0, ignore_index = True)#concatinating along axis 0
    return features_df
    #lst_columns=['Position','Track Name','Artist','URL','filename','ID','date','country','group','duration_ms', 'popularity', 'acousticness','danceability', 'energy', 'instrumentalness', 'liveness', 'loudness','mode', 'speechiness', 'valence', 'tempo','loud_start']
    #df_with_features=pd.concat([df,features_df],axis=1,ignore_index = True)
    #df_with_features.columns=[list(df.columns)+list(features_df.columns)]#creates new joined list of columns
    #return df_with_features

#### call API on IDs returning a dataframe with features of each ID

In [ ]:
#dataframe of features
viral_features_df=getTrackFeatures(viral_df_basic_top20) #get features for each song


In [36]:
print(len(viral_features_df))

# null values left
print(viral_features_df.isnull().sum())
viral_features_df.head()

5872
ID                  0
length              0
popularity          0
acousticness        0
danceability        0
energy              0
key                 0
instrumentalness    0
liveness            0
loudness            0
mode                0
speechiness         0
valence             0
tempo               0
loud_start          0
dtype: int64


,ID,length,popularity,acousticness,danceability,energy,key,instrumentalness,liveness,loudness,mode,speechiness,valence,tempo,loud_start
0,5U3ln41nbhztweY70Mv5b3,179079,57,0.1700,0.573,0.722,2,0,0.1100,-5.020,1,0.2520,0.198,168.441,-18.286
1,5sGb7vB83MhpdZsx1tEIYH,139500,0,0.0365,0.880,0.730,6,6.33e-06,0.2320,-7.008,1,0.1810,0.860,159.957,-9.230
2,2A2Q4CYBcVvBF2OqgJ3OWX,192975,51,0.1200,0.846,0.467,2,7.6e-05,0.0803,-7.178,1,0.1730,0.367,129.016,-16.712
3,7Fg342AJtNsIDdwCfX0paC,185702,1,0.2390,0.460,0.567,9,6.04e-06,0.0934,-5.637,1,0.0515,0.189,168.419,-7.641
4,3lCbsHaN1wCxyDzcNN2x4N,182198,84,0.1850,0.720,0.613,1,0,0.1550,-6.899,0,0.0572,0.857,84.974,-13.644


# Leftjoin 
left join the 13282 charts entries with the dataframe of the unique 5872 music features

In [40]:
viral_df=viral_df_basic_top20.merge(viral_features_df, how='left',on='ID')


Position             0
Track Name          11
Artist              12
URL                  0
filename             0
ID                   0
date                 0
country              0
group                0
length               5
popularity           5
acousticness         5
danceability         5
energy               5
key                  5
instrumentalness     5
liveness             5
loudness             5
mode                 5
speechiness          5
valence              5
tempo                5
loud_start           5
dtype: int64

In [48]:
#drop rows with missing music data - most notably Baccara "Yes Sir I can boogey"
viral_df.dropna(axis=0, how='any', thresh=17, inplace=True)
len(viral_df)

13277

# EXPORT TO CSV
Analysis will happen in jupyter notebook 'analysis'

In [ ]:
viral_df.rename(columns={'length':'duration_ms'})

In [49]:
viral_df.to_csv(r'data/analysis/viral_charts_songs_2017-2020.csv', index = False)

In [398]:
###########################################